In [ ]:
import os

import numpy as np

import tensorflow as tf

import matplotlib.pyplot as plt

import torch

import keras
from keras.models import Model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, BackupAndRestore, TensorBoard
from keras.layers import Input, LSTM, Dense, Dropout, Permute, Lambda, Multiply, Activation, GRU, Flatten, Conv1D, MaxPooling1D, Flatten, Average, BatchNormalization, Conv2D, MaxPooling2D

from sklearn.metrics import confusion_matrix, precision_recall_fscore_support,mean_squared_error

import keras_tuner as kt

%load_ext tensorboard

In [ ]:
# Using my GPU to accelrate the results
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
shapes = np.load('shapes.npy',allow_pickle=True)
X_shape = shapes.item()['X']
y_shape = shapes.item()['y']

In [ ]:
X = np.memmap('X_sequence.dat', dtype='float64', mode='r+', shape=X_shape)
Y = np.memmap('Y_sequence.dat', dtype='float64', mode='r+', shape=y_shape)

In [ ]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, X_path, Y_path, shapes , indices, batch_size=32,shuffle=True):

        self.X = np.memmap(X_path, dtype='float64', mode='r+', shape=shapes.item()['X'])
        self.Y = np.memmap(Y_path, dtype='float64', mode='r+', shape=shapes.item()['y'])

        self.indices = indices
        self.num_samples = len(self.indices)

        self.batch_size = batch_size
        
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(self.num_samples / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        X = self.X[batch_indices]
        Y = self.Y[batch_indices]
        return X,Y

    def on_epoch_end(self):
        if self.shuffle == True:
            np.random.shuffle(self.indices)

In [ ]:
def LSTM_Model(input_size, output_shape, hidden_size=64, dropout=0.2):
    inputs = Input(shape=(input_size[1], input_size[2])) # (batch_size, time_step, features)
    
    # LSTM layers
    lstm_out = LSTM(hidden_size, return_sequences=True)(inputs)
    lstm_out = Dropout(dropout)(lstm_out)

    lstm_out = LSTM(int(hidden_size*2), return_sequences=True)(lstm_out)
    lstm_out = Dropout(dropout+0.1)(lstm_out)

    lstm_out = LSTM(int(hidden_size*4), return_sequences=True)(lstm_out)
    lstm_out = Dropout(dropout+0.2)(lstm_out)

    # Attention mechanism
    attn_weights = Dense(1, activation='tanh')(lstm_out)
    attn_weights = Permute((2, 1))(attn_weights)
    attn_weights = Activation('softmax')(attn_weights)
    attn_weights = Permute((2, 1))(attn_weights)
    attn_output = Multiply()([attn_weights, lstm_out])
    context_vector = Lambda(lambda x: keras.backend.sum(x, axis=1))(attn_output)
    
    # Fully connected layers
    fc1 = Dense(int(hidden_size), activation='relu')(context_vector)
    fc1 = Dropout(dropout)(fc1)
    fc2 = Dense(int(output_shape[1] * 2), activation='relu')(fc1)
    fc2 = Dropout(dropout)(fc2)
    predictions = Dense(output_shape[1], activation='linear')(fc2)
    
    model = Model(inputs=inputs, outputs=predictions)
    return model

In [ ]:
def GRU_Model(input_size, output_shape, hidden_size=64, dropout=0.2):
    inputs = Input(shape=(input_size[1], input_size[2])) # (batch_size, time_step, features)

    # GRU layers
    gru_out = GRU(hidden_size, activation='tanh',return_sequences=True)(inputs)
    gru_out = Dropout(dropout)(gru_out)

    gru_out = GRU(int(hidden_size/2), activation='tanh',return_sequences=True)(gru_out)
    gru_out = Dropout(dropout+0.1)(gru_out)
    
    gru_out = Flatten()(gru_out)

    fc1 = Dense(hidden_size, activation='relu')(gru_out)
    fc1 = Dropout(dropout)(fc1)
    fc2 = Dense(int(output_shape[1] * 2), activation='relu')(fc1)
    fc2 = Dropout(dropout)(fc2)
    predictions = Dense(output_shape[1], activation='linear')(fc2)
    
    model = Model(inputs=inputs, outputs=predictions)
    return model

In [ ]:
def CNN_Model(input_size, output_shape, hidden_size=64, dropout=0.2,kernel_size=3, pool_size=2):
    inputs = Input(shape=(input_size[1], input_size[2])) # (batch_size, time_step, features)

    # CNN layers
    x = Conv1D(filters=hidden_size, kernel_size=kernel_size, activation='relu')(inputs)
    x = MaxPooling1D(pool_size=pool_size)(x)

    x = Conv1D(filters=int(hidden_size/2), kernel_size=kernel_size, activation='relu')(x)
    x = MaxPooling1D(pool_size=pool_size)(x)

    x = Flatten()(x)

    fc1 = Dense(hidden_size, activation='relu')(x)
    fc1 = Dropout(dropout)(fc1)
    fc2 = Dense(int(output_shape[1] * 2), activation='relu')(fc1)
    fc2 = Dropout(dropout)(fc2)
    predictions = Dense(output_shape[1], activation='linear')(fc2)
    
    model = Model(inputs=inputs, outputs=predictions)
    return model

In [ ]:
def NN_Model(input_size, output_shape, hidden_size=64, dropout=0.2):
    inputs = Input(shape=(input_size[1], input_size[2])) # (batch_size, time_step, features)

    x = Dense(int(hidden_size*2), activation='relu')(inputs)
    x = BatchNormalization()(x)

    x = Dropout(dropout)(x)

    x = Dense(hidden_size, activation='relu')(x)
    x = BatchNormalization()(x)

    x = Dropout(dropout+0.1)(x)

    x = Flatten()(x)

    fc1 = Dense(int(hidden_size), activation='relu')(x)
    fc1 = Dropout(dropout)(fc1)

    fc2 = Dense(int(output_shape[1] * 2), activation='relu')(fc1)
    fc2 = Dropout(dropout)(fc2)

    predictions = Dense(output_shape[1])(fc2)
    
    model = Model(inputs=inputs, outputs=predictions)
    return model

In [ ]:
# Define model parameters
INPUT_SIZE = X_shape
HIDDEN_SIZE = 180
OUTPUT_SIZE = y_shape
NUM_LAYERS = 6

# Define model architecture
KERNEL_SIZE = 3
POOL_SIZE = 2
DROPOUT = 0.1
N_ESTIMATORS = 100

# Define training parameters
LEARNING_RATE = 0.001
BATCH_SIZE = 30
NUM_EPOCHS = 40
GAMMA = 0.5
STEP_SIZE = 3
PATIENCE = 10
MAX_TRIALS = 50
SHUFFLE = True
CHECKPOINT_DIR = "./checkpoint_keras"
BACKUP_DIR = "./backup_keras"
LOG_DIR = "./logs_keras"

# DATA SPLIT
TEST_SPLIT = 0.15
VAL_SPLIT = 0.15
TRAIN_SPLIT = 1 - TEST_SPLIT - VAL_SPLIT

# Define random seed
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [ ]:
num_samples = Y.shape[0]

# Shuffle the data indices
indices = np.arange(num_samples)
np.random.shuffle(indices)

num_validation_samples = int(num_samples * VAL_SPLIT)
num_test_samples = int(num_samples * TEST_SPLIT)
num_train_samples = num_samples - num_validation_samples - num_test_samples

train_indices = indices[:num_train_samples]
val_indices = indices[num_train_samples:num_train_samples + num_validation_samples]
test_indices = indices[num_train_samples + num_validation_samples:]

train_params = {'batch_size': BATCH_SIZE,
                'shuffle': SHUFFLE,
                'X_path': 'X_sequence.dat',
                'Y_path': 'Y_sequence.dat',
                'shapes': shapes,
                'indices': train_indices}

val_params = {'batch_size': BATCH_SIZE,
              'shuffle': SHUFFLE,
              'X_path': 'X_sequence.dat',
              'Y_path': 'Y_sequence.dat',
              'shapes': shapes,
              'indices': val_indices}

test_params = {'batch_size': BATCH_SIZE,
               'shuffle': SHUFFLE,
               'X_path': 'X_sequence.dat',
               'Y_path': 'Y_sequence.dat',
               'shapes': shapes,
               'indices': test_indices}

train_dataset = DataGenerator(**train_params)
val_dataset = DataGenerator(**val_params)
test_dataset = DataGenerator(**test_params)

In [ ]:
X_train, y_train = next(iter(train_dataset))
print(X_train.shape, y_train.shape)

In [ ]:
# Function to create LSTM model
def create_lstm_model(input_shape, output_size, hidden_size, dropout, optimizer, loss):
    model = LSTM_Model(input_size=input_shape, output_shape=output_size, hidden_size=hidden_size, dropout=dropout)
    model.compile(optimizer=optimizer, loss=loss)
    model._name = 'LSTM_Model'
    print(model.summary())
    return model

# Function to create GRU model
def create_gru_model(input_shape, output_size, hidden_size, dropout, optimizer, loss):
    model = GRU_Model(input_size=input_shape, output_shape=output_size, hidden_size=hidden_size, dropout=dropout)
    model._name = 'GRU_Model'
    model.compile(optimizer=optimizer, loss=loss)
    print(model.summary())
    return model

# Function to create CNN model
def create_cnn_model(input_shape, output_size, hidden_size, dropout, kernel_size, pool_size, optimizer, loss):
    model = CNN_Model(input_size=input_shape, output_shape=output_size, hidden_size=hidden_size, dropout=dropout, kernel_size=kernel_size, pool_size=pool_size)
    model._name = 'CNN_Model'
    model.compile(optimizer=optimizer, loss=loss)
    print(model.summary())
    return model

# Function to create Random Forest model
def create_nn_model(input_shape, output_size, hidden_size, dropout, optimizer, loss):
    model = NN_Model(input_size=input_shape, output_shape=output_size, hidden_size=hidden_size, dropout=dropout)
    model._name = 'NN_Model'
    model.compile(optimizer=optimizer, loss=loss)
    print(model.summary())
    return model

# Function to create ensemble model
def create_ensemble_model(input_shape, output_size, hidden_size, dropout, kernel_size, pool_size, optimizer, loss):
    lstm_model = create_lstm_model(input_shape, output_size, hidden_size, dropout, optimizer, loss)
    gru_model = create_gru_model(input_shape, output_size, hidden_size, dropout, optimizer, loss)
    cnn_model = create_cnn_model(input_shape, output_size, hidden_size, dropout, kernel_size, pool_size, optimizer, loss)
    nn_model = create_nn_model(input_shape, output_size, hidden_size, dropout, optimizer, loss)
    models = [lstm_model, gru_model, cnn_model, nn_model]
    input = Input(shape=(input_shape[1], input_shape[2]))
    outputs = [model(input) for model in models]
    ensemble_output = Average()(outputs)
    ensemble_output = Dense(output_size[1], activation='linear')(ensemble_output)
    ensemble_model = Model(inputs=input, outputs=ensemble_output)
    ensemble_model._name = 'Ensemble_Model'
    return ensemble_model

In [ ]:
from keras import losses, optimizers

OPTIMIZER = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
LOSS = losses.MeanSquaredError()

ensemble_model = create_ensemble_model(input_shape=INPUT_SIZE, output_size=OUTPUT_SIZE, hidden_size=HIDDEN_SIZE, dropout=DROPOUT, kernel_size=KERNEL_SIZE, pool_size=POOL_SIZE, optimizer=OPTIMIZER, loss=LOSS)
ensemble_model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=[keras.metrics.RootMeanSquaredError(), keras.metrics.MeanAbsoluteError(), keras.metrics.MeanAbsolutePercentageError(), keras.metrics.MeanSquaredLogarithmicError(), keras.metrics.CosineSimilarity(axis=1, name='cosine_similarity')])
print(ensemble_model.summary())

In [ ]:
early_stopping = EarlyStopping(patience=PATIENCE, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(factor=GAMMA, patience=STEP_SIZE)

checkpoint_filepath = CHECKPOINT_DIR + '/checkpoint_RoboStockModel_{epoch:02d}-{val_accuracy:.2f}.keras'
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_accuracy', mode='max', save_best_only=True, overwrite=True)

backup_callback = BackupAndRestore(backup_dir=BACKUP_DIR)

tensorboard_callback = TensorBoard(log_dir=LOG_DIR)

In [ ]:
import shutil

torch.cuda.empty_cache()
if os.path.exists(LOG_DIR):
    shutil.rmtree(LOG_DIR)

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir {LOG_DIR} --host localhost --port 8888
history = ensemble_model.fit(train_dataset, epochs=NUM_EPOCHS, steps_per_epoch=len(train_dataset), validation_data=val_dataset, validation_steps=len(val_dataset), callbacks=[backup_callback,early_stopping, reduce_lr, checkpoint_callback, tensorboard_callback], shuffle=SHUFFLE)

In [ ]:
X_test, y_test = next(iter(test_dataset))

output = ensemble_model.predict(X_test, batch_size=BATCH_SIZE, verbose=1)


# Evaluate the model
mse = mean_squared_error(y_test, output)
print("Mean Squared Error:", mse)

test_loss = ensemble_model.evaluate(test_dataset, steps=len(test_dataset))
print("Test Loss:", test_loss)

In [ ]:
ensemble_model.save('RoboStockModel.keras')

In [ ]:
ensemble_model.save_weights('./checkpoint/RoboStockModel_weights')

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:
y_len = y_test.shape[0]

fig, axs = plt.subplots(int(y_len/2),y_len-int(y_len/2),figsize=(50, 50))
for i in range(int(y_len/2)):
    for j in range(y_len-int(y_len/2)):
        axs[i][j].scatter(y_test[i+j], output[i+j])
fig.suptitle('y_true vs y_pred')
fig.tight_layout()
fig.subplots_adjust(top=0.95)
plt.show()

In [ ]:
y_len = y_test.shape[0]

fig, axs = plt.subplots(int(y_len/2),y_len-int(y_len/2),figsize=(50, 50))
X_axis = np.arange(0, len(y_test[0]), 1)
for i in range(int(y_len/2)):
    for j in range(y_len-int(y_len/2)):
        axs[i][j].plot(X_axis,y_test[i+j], color = 'red', label = 'y_test')
        axs[i][j].plot(X_axis,output[i+j], color = 'blue', label = 'y_pred', linestyle = 'dashed')
fig.suptitle('y_true vs y_pred')
fig.tight_layout()
fig.subplots_adjust(top=0.95)
plt.show()